In [17]:
#pip install transformers

In [18]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import BertTokenizer, BertModel
import torch

In [19]:
ratings = pd.read_csv('ratings_formatted.csv')

In [20]:

comments_short = ratings['summary'].dropna().head(100000).values

In [21]:
comments_short

array(['Nice collection of Julie Strain images', 'Really Enjoyed It',
       'Essential for every personal and Public Library', ...,
       'Highly suggested for beginners',
       'Good for the beginner, nice paper for the experienced',
       'Greate Activity Book for Kids'], dtype=object)

# Mots simples

In [22]:
size_vocabulary = 1000
vectorizer = CountVectorizer(stop_words = "english", max_features = size_vocabulary)

In [23]:
X = vectorizer.fit_transform(comments_short)
X.toarray().shape

(100000, 1000)

# TF IDF

In [24]:
size_vocabulary = 1000
vectorizer = TfidfVectorizer(stop_words = "english", max_features = size_vocabulary)

In [25]:
X = vectorizer.fit_transform(comments_short)
X.toarray().shape

(100000, 1000)

In [26]:
vectorizer.get_feature_names_out()

array(['000', '10', '100', '11', '12', '1491', '1984', '20', '20th', '34',
       '451', '65533', 'absolute', 'absolutely', 'account', 'accurate',
       'action', 'actually', 'addition', 'adult', 'adults', 'adventure',
       'adventures', 'advice', 'age', 'ages', 'ago', 'agree', 'ahead',
       'alive', 'amazing', 'amazon', 'america', 'american', 'americans',
       'amp', 'amusing', 'analysis', 'ancient', 'anger', 'anti',
       'approach', 'april', 'art', 'asimov', 'attention', 'audio',
       'audiobook', 'auel', 'austen', 'author', 'authors',
       'autobiography', 'available', 'average', 'away', 'awesome',
       'awful', 'awsome', 'ayla', 'baby', 'bad', 'baseball', 'based',
       'basic', 'basics', 'bear', 'beautiful', 'beautifully', 'beauty',
       'beginner', 'beginners', 'beginning', 'believe', 'best', 'better',
       'beware', 'biased', 'bible', 'biblical', 'big', 'bilbo',
       'biography', 'bit', 'black', 'blackbird', 'boleyn', 'book',
       'books', 'boring', 'both

# Bert embbeding

In [27]:
    
def get_bert_latent_representation(text,model,tokenizer):
   
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    
    cls_embedding = outputs.last_hidden_state[:, 0, :]
    
    return cls_embedding

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
sample_text = comments_short[0]
latent_representation = get_bert_latent_representation(sample_text,model,tokenizer)
print("Latent representation shape:", latent_representation.shape)

Latent representation shape: torch.Size([1, 768])
